In [4]:
"""
REPRODUCTION DES 4 SCÉNARIOS DE SIMULATION DE L'ARTICLE RLT
Section 4.2 - Simulation Scenarios

Chaque scénario est répété 200 fois avec p ∈ {200, 500, 1000}
"""

import numpy as np
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.metrics import mean_squared_error, accuracy_score
from scipy.stats import norm
import pandas as pd
import matplotlib.pyplot as plt
import time

# Importer la classe RLT du fichier précédent
# from rlt_implementation import RLT

def generate_scenario_1(N, p, seed=None):
    
    Scénario 1 : Classification avec covariables indépendantes
    
    N = 100
    X ~ Uniform[0,1]^p
    μ = Φ(10*(X₁ - 1) + 20*|X₂ - 0.5|)
    Y ~ Bernoulli(μ)
    
    Variables importantes : X₁, X₂
    
    if seed is not None:
        np.random.seed(seed)
    
    X = np.random.uniform(0, 1, size=(N, p))
    
    # Calculer μ
    mu = norm.cdf(10 * (X[:, 0] - 1) + 20 * np.abs(X[:, 1] - 0.5))
    
    # Générer Y ~ Bernoulli(μ)
    Y = np.random.binomial(1, mu)
    
    return X, Y

def generate_scenario_2(N, p, seed=None):
    
    Scénario 2 : Modèle non-linéaire avec covariables indépendantes
    
    N = 100
    X ~ Uniform[0,1]^p
    Y = 100(X₁ - 0.5)² × (X₂ - 0.25)₊ + ε
    ε ~ N(0,1)
    
    Variables importantes : X₁, X₂
    Note : (x)₊ = max(x, 0)
    
    if seed is not None:
        np.random.seed(seed)
    
    X = np.random.uniform(0, 1, size=(N, p))
    
    # Calculer Y
    Y = 100 * (X[:, 0] - 0.5)**2 * np.maximum(X[:, 1] - 0.25, 0)
    Y += np.random.normal(0, 1, N)
    
    return X, Y

def generate_scenario_3(N, p, seed=None):
    
    Scénario 3 : Modèle checkerboard avec forte corrélation
    
    N = 300
    X ~ N(0, Σ) où Σᵢⱼ = 0.9^|i-j|
    Y = 2X₅₀×X₁₀₀ + 2X₁₅₀×X₂₀₀ + ε
    ε ~ N(0,1)
    
    Variables importantes : X₅₀, X₁₀₀, X₁₅₀, X₂₀₀
    
    if seed is not None:
        np.random.seed(seed)
    
    # Créer matrice de covariance : Σᵢⱼ = 0.9^|i-j|
    Sigma = np.zeros((p, p))
    for i in range(p):
        for j in range(p):
            Sigma[i, j] = 0.9 ** abs(i - j)
    
    # Générer X ~ N(0, Σ)
    X = np.random.multivariate_normal(np.zeros(p), Sigma, size=N)
    
    # Calculer Y (attention : indices Python commencent à 0)
    Y = 2 * X[:, 49] * X[:, 99] + 2 * X[:, 149] * X[:, 199]
    Y += np.random.normal(0, 1, N)
    
    return X, Y

def generate_scenario_4(N, p, seed=None):
    
    Scénario 4 : Modèle linéaire
    
    N = 200
    X ~ N(0, Σ) où Σᵢⱼ = 0.5^|i-j| + 0.2×I(i,j)
    Y = 2X₅₀ + 2X₁₀₀ + 4X₁₅₀ + ε
    ε ~ N(0,1)
    
    Variables importantes : X₅₀, X₁₀₀, X₁₅₀
    
    if seed is not None:
        np.random.seed(seed)
    
    # Créer matrice de covariance : Σᵢⱼ = 0.5^|i-j| + 0.2×I(i,j)
    Sigma = np.zeros((p, p))
    for i in range(p):
        for j in range(p):
            Sigma[i, j] = 0.5 ** abs(i - j)
            if i == j:
                Sigma[i, j] += 0.2
    
    # Générer X ~ N(0, Σ)
    X = np.random.multivariate_normal(np.zeros(p), Sigma, size=N)
    
    # Calculer Y (indices Python commencent à 0)
    Y = 2 * X[:, 49] + 2 * X[:, 99] + 4 * X[:, 149]
    Y += np.random.normal(0, 1, N)
    
    return X, Y

"""
def run_single_simulation(scenario_func, scenario_params, model_func, 
                         n_test=1000, seed=None):
    
    Exécute une simulation unique
    
    Returns:
        test_error: erreur sur données de test
        train_error: erreur sur données d'entraînement
        time: temps d'exécution
    
    start_time = time.time()
    
    # Générer données d'entraînement
    X_train, Y_train = scenario_func(**scenario_params, seed=seed)
    
    # Générer données de test (même distribution, seed différent)
    test_seed = seed + 100000 if seed is not None else None
    X_test, Y_test = scenario_func(
        N=n_test, 
        p=scenario_params['p'], 
        seed=test_seed
    )
    
    # Entraîner le modèle
    model = model_func()
    model.fit(X_train, Y_train)
    
    # Prédictions
    pred_train = model.predict(X_train)
    pred_test = model.predict(X_test)
    
    # Calculer erreurs
    if hasattr(scenario_func, '__name__') and 'scenario_1' in scenario_func.__name__:
        # Classification
        train_error = 1 - accuracy_score(Y_train, pred_train > 0.5)
        test_error = 1 - accuracy_score(Y_test, pred_test > 0.5)
    else:
        # Régression
        train_error = mean_squared_error(Y_train, pred_train)
        test_error = mean_squared_error(Y_test, pred_test)
    
    elapsed_time = time.time() - start_time
    
    return test_error, train_error, elapsed_time


def run_experiment(scenario_name, scenario_func, scenario_params,
                   models_dict, n_repetitions=200, n_test=1000):
    
    Exécute une expérience complète pour un scénario donné
    
    Parameters:
        scenario_name: nom du scénario
        scenario_func: fonction pour générer les données
        scenario_params: paramètres du scénario (N, p)
        models_dict: dictionnaire {nom_modèle: fonction_créer_modèle}
        n_repetitions: nombre de répétitions (200 dans l'article)
        n_test: taille échantillon de test (1000 dans l'article)
    
    Returns:
        DataFrame avec résultats
    
    print(f"\n{'='*70}")
    print(f"Scénario: {scenario_name}")
    print(f"N={scenario_params['N']}, p={scenario_params['p']}")
    print(f"{'='*70}")
    
    results = []
    
    for model_name, model_func in models_dict.items():
        print(f"\n{model_name}...")
        
        test_errors = []
        train_errors = []
        times = []
        
        for rep in range(n_repetitions):
            try:
                test_err, train_err, elapsed = run_single_simulation(
                    scenario_func,
                    scenario_params,
                    model_func,
                    n_test=n_test,
                    seed=rep
                )
                
                test_errors.append(test_err)
                train_errors.append(train_err)
                times.append(elapsed)
                
                if (rep + 1) % 50 == 0:
                    mean_err = np.mean(test_errors)
                    print(f"  {rep+1}/{n_repetitions} - Erreur moyenne: {mean_err:.4f}")
                    
            except Exception as e:
                print(f"  Erreur à la répétition {rep}: {e}")
                continue
        
        # Calculer statistiques
        mean_test = np.mean(test_errors)
        std_test = np.std(test_errors)
        mean_time = np.mean(times)
        
        results.append({
            'Model': model_name,
            'Mean_Test_Error': mean_test,
            'Std_Test_Error': std_test,
            'Mean_Time': mean_time,
            'N_Success': len(test_errors)
        })
        
        print(f"  → Erreur Test: {mean_test:.4f} ± {std_test:.4f}")
        print(f"  → Temps moyen: {mean_time:.2f}s")
    
    df_results = pd.DataFrame(results)
    return df_results

# ============================================
# CONFIGURATION DES MODÈLES À COMPARER
# ============================================

def create_models_dict(p, task='regression'):
    
    Crée le dictionnaire des modèles à comparer
    
    models = {}
    
    # Random Forest classique
    if task == 'regression':
        models['RF'] = lambda: RandomForestRegressor(
            n_estimators=500,
            max_features='sqrt',
            min_samples_leaf=2,
            random_state=42
        )
    else:
        models['RF'] = lambda: RandomForestClassifier(
            n_estimators=500,
            max_features='sqrt',
            min_samples_leaf=2,
            random_state=42
        )
 """   
    # Pour utiliser RLT, décommenter ci-dessous (nécessite le code précédent)
    """
    from rlt_implementation import RLT
    
    # RLT avec différentes configurations
    models['RLT (k=1, no muting)'] = lambda: RLT(
        n_trees=100,
        muting_rate=0.0,
        k=1,
        task=task
    )
    
    models['RLT (k=1, moderate)'] = lambda: RLT(
        n_trees=100,
        muting_rate=0.5,
        k=1,
        task=task
    )
    
    models['RLT (k=2, aggressive)'] = lambda: RLT(
        n_trees=100,
        muting_rate=0.8,
        k=2,
        task=task
    )
    
    
    return models

"""
# ============================================
# EXEMPLE D'EXÉCUTION
# ============================================

if __name__ == "__main__":
    print("="*70)
    print("REPRODUCTION DES SIMULATIONS DE L'ARTICLE RLT")
    print("="*70)
    
    # Choisir un scénario à tester (pour démo rapide)
    scenario_to_test = 1  # Change ce numéro pour tester différents scénarios
    p_to_test = 200       # Dimensions à tester : 200, 500, ou 1000
    n_reps_demo = 200      # Réduit pour démo (mettre 200 pour vraie simulation)
    
    scenarios = {
        1: {
            'name': 'Scénario 1 - Classification',
            'func': generate_scenario_1,
            'params': {'N': 100, 'p': p_to_test},
            'task': 'classification'
        },
        2: {
            'name': 'Scénario 2 - Non-linéaire',
            'func': generate_scenario_2,
            'params': {'N': 100, 'p': p_to_test},
            'task': 'regression'
        },
        3: {
            'name': 'Scénario 3 - Checkerboard',
            'func': generate_scenario_3,
            'params': {'N': 300, 'p': p_to_test},
            'task': 'regression'
        },
        4: {
            'name': 'Scénario 4 - Linéaire',
            'func': generate_scenario_4,
            'params': {'N': 200, 'p': p_to_test},
            'task': 'regression'
        }
    }
    
    # Sélectionner le scénario
    scenario = scenarios[scenario_to_test]
    
    # Créer les modèles à comparer
    #models_dict = create_models_dict(p_to_test, task=scenario['task'])
    
    # Lancer l'expérience
    results_df = run_experiment(
        scenario_name=scenario['name'],
        scenario_func=scenario['func'],
        scenario_params=scenario['params'],
        #models_dict=models_dict,
        n_repetitions=n_reps_demo,
        n_test=1000
    )
    
    # Afficher résultats
    print("\n" + "="*70)
    print("RÉSULTATS FINAUX")
    print("="*70)
    print(results_df.to_string(index=False))
    '''
    # Sauvegarder résultats
    output_file = f"datasets_augmented/results_scenario{scenario_to_test}_p{p_to_test}.csv"
    results_df.to_csv(output_file, index=False)
    print(f"\nRésultats sauvegardés dans: {output_file}")
    
    # Créer graphique comparatif
    plt.figure(figsize=(10, 6))
    plt.bar(results_df['Model'], results_df['Mean_Test_Error'], 
            yerr=results_df['Std_Test_Error'], capsize=5)
    plt.xlabel('Modèle')
    plt.ylabel('Erreur de Test Moyenne')
    plt.title(f"{scenario['name']} (p={p_to_test})")
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig(f"plot_scenario{scenario_to_test}_p{p_to_test}.png", dpi=300)
    print(f"Graphique sauvegardé: plot_scenario{scenario_to_test}_p{p_to_test}.png")
    
    print("\n✅ Simulation terminée!")
    '''
     X, Y = scenario_functions[scenario_to_generate](N=N, p=p_to_generate, seed=seeds)
    
    # Sauvegarder le dataset généré
    df = pd.DataFrame(X, columns=[f"X{i+1}" for i in range(p_to_generate)])
    df['Y'] = Y
    output_file = f"datasets_augmented/scenario{scenario_to_generate}_p{p_to_generate}.csv"
    df.to_csv(output_file, index=False)
    print(f"✅ Dataset généré et sauvegardé dans: {output_file}")

SyntaxError: invalid character '₁' (U+2081) (3631554371.py, line 25)

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import norm

# =====================
# Fonctions des scénarios
# =====================

def generate_scenario_1(N, p, seed=None):
    if seed is not None:
        np.random.seed(seed)
    X = np.random.uniform(0, 1, size=(N, p))
    mu = norm.cdf(10 * (X[:, 0] - 1) + 20 * np.abs(X[:, 1] - 0.5))
    Y = np.random.binomial(1, mu)
    return X, Y

def generate_scenario_2(N, p, seed=None):
    if seed is not None:
        np.random.seed(seed)
    X = np.random.uniform(0, 1, size=(N, p))
    Y = 100 * (X[:, 0] - 0.5)**2 * np.maximum(X[:, 1] - 0.25, 0)
    Y += np.random.normal(0, 1, N)
    return X, Y

def generate_scenario_3(N, p, seed=None):
    if seed is not None:
        np.random.seed(seed)
    Sigma = np.array([[0.9 ** abs(i - j) for j in range(p)] for i in range(p)])
    X = np.random.multivariate_normal(np.zeros(p), Sigma, size=N)
    Y = 2 * X[:, 49] * X[:, 99] + 2 * X[:, 149] * X[:, 199]
    Y += np.random.normal(0, 1, N)
    return X, Y

def generate_scenario_4(N, p, seed=None):
    if seed is not None:
        np.random.seed(seed)
    Sigma = np.array([[0.5 ** abs(i - j) + (0.2 if i==j else 0) for j in range(p)] for i in range(p)])
    X = np.random.multivariate_normal(np.zeros(p), Sigma, size=N)
    Y = 2 * X[:, 49] + 2 * X[:, 99] + 4 * X[:, 149]
    Y += np.random.normal(0, 1, N)
    return X, Y

# =====================
# Exemple de génération
# =====================

scenario_functions = {
    1: generate_scenario_1,
    2: generate_scenario_2,
    3: generate_scenario_3,
    4: generate_scenario_4
}

# Choisir scénario et paramètres
scenario_to_generate = 1   # 1 à 4
N = 100
p = 200
seed = 42

X, Y = scenario_functions[scenario_to_generate](N=N, p=p, seed=seed)

# Sauvegarde en CSV
df = pd.DataFrame(X, columns=[f"X{i+1}" for i in range(p)])
df['Y'] = Y
output_file = f"datasets_augmented/scenario{scenario_to_generate}_p{p}.csv"
df.to_csv(output_file, index=False)
print(f"Dataset généré et sauvegardé dans {output_file}")
